In [ ]:
import pandas as pd
import numpy as np
import sklearn
dataset = pd.read_csv("/content/sms_spam_dataset.csv")
x = dataset["message"]
y = dataset["label"]

In [ ]:
dataset.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
dataset.drop_duplicates(inplace= True)
dataset.dropna(inplace= True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset['label'] = le.fit_transform(dataset['label'])


In [ ]:
dataset.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import string
def processing(message):
  no_punc= [char for char in message if char not in string.punctuation]
  no_punc= ''.join(no_punc)

  clean_words= [word for word in no_punc.split() if word.lower() not in stopwords.words('english')]
  return clean_words

In [ ]:
dataset['message'].apply(processing)

,message
0,"[Go, jurong, point, crazy, Available, bugis, n..."
1,"[Ok, lar, Joking, wif, u, oni]"
2,"[Free, entry, 2, wkly, comp, win, FA, Cup, fin..."
3,"[U, dun, say, early, hor, U, c, already, say]"
4,"[Nah, dont, think, goes, usf, lives, around, t..."
...,...
5567,"[2nd, time, tried, 2, contact, u, U, £750, Pou..."
5568,"[ü, b, going, esplanade, fr, home]"
5569,"[Pity, mood, Soany, suggestions]"
5570,"[guy, bitching, acted, like, id, interested, b..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)
x_tfidf = vectorizer.fit_transform(dataset['message']).toarray()

In [ ]:
y = dataset['label'].values

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_tfidf, y, test_size=0.2, random_state=0)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report, accuracy_score

model = Sequential([
    Dense(512, activation='relu', input_shape=(x_train.shape[1],)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - loss: 0.3509 - val_loss: 0.0955
Epoch 2/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - loss: 0.0403 - val_loss: 0.0807
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - loss: 0.0113 - val_loss: 0.0814
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0047 - val_loss: 0.0888
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 0.0014 - val_loss: 0.0959
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 8.2722e-04 - val_loss: 0.0986
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0019 - val_loss: 0.1021
Epoch 8/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - loss: 0.0023 - val_loss: 0.1045
Epoch 9/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - loss: 0.0014 - val_loss: 0.1048
Epoch 10/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 3.9635e-04 - val_loss: 0.1089


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, mean_squared_error, mean_absolute_error
)

y_pred_prob = model.predict(x_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\n")
print("Precision:", precision_score(y_test, y_pred))
print("\n")
print("Recall:", recall_score(y_test, y_pred))
print("\n")
print("F1 Score:", f1_score(y_test, y_pred))
print("\n")
print("MSE:", mean_squared_error(y_test, y_pred_prob))
print("\n")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9787234042553191


Precision: 0.9202898550724637


Recall: 0.9202898550724637


F1 Score: 0.9202898550724637


MSE: 0.019910652190446854


Confusion Matrix:
 [[885  11]
 [ 11 127]]


In [ ]:
def predict_message(message):
    cleaned_msg = message.lower()
    msg_vector = vectorizer.transform([cleaned_msg]).toarray()
    prediction_prob = model.predict(msg_vector)[0][0]
    prediction = "SPAM" if prediction_prob > 0.5 else "NOT SPAM"
    print(f"Prediction: {prediction}")


In [ ]:
user_msg = input("Enter a message: ")
predict_message(user_msg)